## Video Game Character Project
#### Part II:
Apply what you have done in the previous project and learned in the previous lectures to update and delete data related to your video game characters within your MongoDB database.

#### Import Dependencies

In [36]:
import copy
import numpy as np

from datetime import datetime 
from pymongo import MongoClient

#### Player and Items Python Classes 

In [37]:
class Player:
    def __init__(self, name="player", max_health=50, max_energy=25, items=[], gold=0):
        self.name = name
        self.health = max_health 
        self.max_health = max_health
        self.energy = max_energy
        self.max_energy = max_energy
        self.items = copy.deepcopy(items)
        self.gold = 0
        
        
    def attack(self, player):
        energy_cost = 5
        
        if self.energy >= energy_cost:
            attack_strength = np.random.randint(7, 13)
            player.health -= attack_strength
            self.energy -= energy_cost
            print("{} attacked {} for {} damage".format(self.name, player.name, attack_strength))
        else:
            print("{} does not have enough energy to attack {}".format(self.name, player.name))
        
        
    def heal(self, amount):
        self.health += amount
        
        if self.health > self.max_health:
            self.health = self.max_health
 
    def restore_energy(self, amount):
        self.energy += amount
        
        if self.energy > self.max_energy:
            self.energy = self.max_energy

    def rest(self):
        self.health = self.max_health
        self.energy = self.max_energy
        
    def stats(self):
        return vars(self)
        
        
    def use_item(self, item_name):
        try: 
            item = next(item for item in self.items if item.name == item_name)
            item.quantity -= 1

            for effect in item.effects:
                for method, value in effect.items():
                    class_method = getattr(self, method)
                    class_method(value)

            print("{} used item: {}".format(self.name, item.name))
                    
            if item.quantity == 0:
                self.items.remove(item)         
                
        except:
            print("{} does not have any {}s".format(self.name, item_name))

In [38]:
class Item:
    def __init__(self, name, quantity, effects=[]):
        self.name = name
        self.quantity = quantity 
        self.effects = effects
        
    def __repr__(self):
        return "Item(name={}, quantity={}, effects={})".format(self.name, self.quantity, self.effects)

In [39]:
class NPC(Player):
    pass

#### Project Instructions

In [40]:
def get_database(url: str, database_name: str):
    """
    Kết nối đến cơ sở dữ liệu MongoDB, nếu chưa có thì tự động tạo.

    Args:
        url (str): Địa chỉ kết nối MongoDB.
        database_name (str): Tên cơ sở dữ liệu cần kết nối.

    Returns:
        Database | None: Đối tượng cơ sở dữ liệu MongoDB hoặc None nếu có lỗi.
    """
    try:
        client = MongoClient(url)
        db = client[database_name]

        # Kiểm tra nếu database chưa tồn tại thì tạo collection mặc định
        if database_name not in client.list_database_names():
            db.create_collection("default_collection")
            print(f"Database '{database_name}' đã được tạo.")
        else:
            print(f"Kết nối thành công đến database '{database_name}'.")

        return db
    except Exception as e:
        print(f"Lỗi khi kết nối đến MongoDB: {e}")
        return None


if __name__ == "__main__":
    URL = "mongodb+srv://testPythonMongo:o@cluster0.ktuns.mongodb.net/"
    DATABASE_NAME = "video_games"

    db = get_database(URL, DATABASE_NAME)



Kết nối thành công đến database 'video_games'.


In [48]:
def convert_player_obj_to_dict(player_obj):
    """
    Chuyển đổi một đối tượng Player thành dictionary để lưu vào cơ sở dữ liệu.

    Args:
        player_obj (Player): Đối tượng người chơi cần chuyển đổi.

    Returns:
        dict: Dictionary chứa thông tin của người chơi, bao gồm danh sách vật phẩm.
    """
    player_dict = copy.deepcopy(vars(player_obj))

    items_dict_list = []
    for item_obj in player_dict["items"]:
        item_dict = vars(item_obj)
        items_dict_list.append(item_dict)

    player_dict["items"] = items_dict_list

    return player_dict


def insert_player_by_obj(player_obj, check_for_duplicates=True):
    """
    Chèn một đối tượng người chơi vào cơ sở dữ liệu.

    Args:
        player_obj (Player): Đối tượng người chơi cần chèn vào cơ sở dữ liệu.
        check_for_duplicates (bool, optional): Nếu True, kiểm tra xem người chơi đã tồn tại chưa.
                                               Mặc định là True.

    Returns:
        ObjectId: ID của bản ghi được chèn vào cơ sở dữ liệu nếu thành công,
                  hoặc ID của bản ghi đã tồn tại nếu có trùng lặp.
    """
    if check_for_duplicates:
        duplicate_player = db.players.find_one({"name": player_obj.name})
        if duplicate_player is not None:
            return duplicate_player["_id"]

    player_dict = convert_player_obj_to_dict(player_obj)
    return db.players.insert_one(player_dict).inserted_id


def find_player_by_name(name):
    """
    Tìm kiếm một người chơi trong cơ sở dữ liệu bằng tên.

    Args:
        name (str): Tên của người chơi cần tìm.

    Returns:
        dict | None: Dictionary chứa thông tin của người chơi nếu tìm thấy,
                     ngược lại trả về None.
    """
    return db.players.find_one({"name": name})


def convert_player_dict_to_obj(player_dict):
    """
    Chuyển đổi một dictionary chứa thông tin người chơi thành một đối tượng Player.

    Args:
        player_dict (dict): Dictionary chứa dữ liệu của người chơi từ cơ sở dữ liệu.

    Returns:
        Player: Đối tượng Player được tạo từ dữ liệu.
    """
    p = player_dict

    items_list = []
    for item in p["items"]:
        item_obj = Item(item["name"], item["quantity"], item["effects"])
        items_list.append(item_obj)

    player_obj = Player()
    player_obj.__dict__ = copy.deepcopy(player_dict)
    player_obj.items = items_list

    return player_obj


def get_player_obj_by_name(name):
    """
    Lấy đối tượng Player từ cơ sở dữ liệu bằng cách tìm kiếm theo tên.

    Args:
        db (Database): Đối tượng kết nối đến cơ sở dữ liệu MongoDB.

    Returns:
        Player | None: Đối tượng Player nếu tìm thấy, ngược lại trả về None.
    """
    player_dict = find_player_by_name(name)
    if player_dict:
        return convert_player_dict_to_obj(player_dict)
    return None


In [49]:
def convert_npc_obj_to_dict(npc_obj):
    """
    Chuyển đổi một đối tượng NPC thành dictionary để lưu vào cơ sở dữ liệu.

    Args:
        npc_obj (NPC): Đối tượng NPC cần chuyển đổi.

    Returns:
        dict: Dictionary chứa thông tin của NPC.
    """
    return copy.deepcopy(vars(npc_obj))


def insert_npc_by_obj(npc_obj, check_for_duplicates=True):
    """
    Chèn một đối tượng NPC vào cơ sở dữ liệu.

    Args:
        npc_obj (NPC): Đối tượng NPC cần chèn vào cơ sở dữ liệu.
        check_for_duplicates (bool, optional): Nếu True, kiểm tra xem NPC đã tồn tại chưa.
                                               Mặc định là True.

    Returns:
        ObjectId: ID của bản ghi được chèn vào cơ sở dữ liệu nếu thành công,
                  hoặc ID của bản ghi đã tồn tại nếu có trùng lặp.
    """
    if check_for_duplicates:
        duplicate_npc = db.npc.find_one({"name": npc_obj.name})
        if duplicate_npc is not None:
            return duplicate_npc["_id"]

    npc_dict = convert_npc_obj_to_dict(npc_obj)
    return db.npc.insert_one(npc_dict).inserted_id


def find_npc_by_name(npc_name):
    """
    Tìm kiếm một NPC trong cơ sở dữ liệu bằng tên.

    Args:
        npc_name (str): Tên của NPC cần tìm.

    Returns:
        dict | None: Dictionary chứa thông tin của NPC nếu tìm thấy,
                     ngược lại trả về None.
    """
    return db.npc.find_one({"name": npc_name})


def get_npc_obj_by_name(npc_name):
    """
    Lấy đối tượng NPC từ cơ sở dữ liệu bằng cách tìm kiếm theo tên.

    Args:
        npc_name (str): Tên của NPC cần tìm.

    Returns:
        NPC | None: Đối tượng NPC nếu tìm thấy, ngược lại trả về None.
    """
    npc_dict = find_npc_by_name(npc_name)
    if npc_dict is None:
        return None

    npc_obj = NPC()
    npc_obj.__dict__ = npc_dict
    return npc_obj


In [50]:
def update_player_by_obj(player_obj):
    """
    Cập nhật thông tin của một người chơi trong cơ sở dữ liệu.

    Args:
        player_obj (Player): Đối tượng người chơi chứa thông tin mới.

    Returns:
        bool: True nếu cập nhật thành công, False nếu không tìm thấy người chơi.
    """
    if not player_obj or not hasattr(player_obj, "name"):
        return False

    player_dict = convert_player_obj_to_dict(player_obj)
    result = db.players.update_one({"name": player_dict["name"]}, {"$set": player_dict})

    return result.modified_count > 0


def delete_player_by_name(player_name):
    """
    Xóa một người chơi khỏi cơ sở dữ liệu bằng tên.

    Args:
        player_name (str): Tên của người chơi cần xóa.

    Returns:
        bool: True nếu xóa thành công, False nếu không tìm thấy người chơi.
    """
    result = db.players.delete_one({"name": player_name})
    return result.deleted_count > 0


def insert_battle_log_by_dict(battle_result_dict):
    """
    Chèn một bản ghi nhật ký trận chiến vào cơ sở dữ liệu.

    Args:
        battle_result_dict (dict): Dictionary chứa thông tin về trận chiến.

    Returns:
        ObjectId: ID của bản ghi được chèn vào cơ sở dữ liệu nếu thành công.
    """
    return db.battle_log.insert_one(battle_result_dict).inserted_id


#### BattleNPC Python Class

In [51]:
class BattleNPC:
    """
    Lớp đại diện cho một trận chiến giữa người chơi và NPC.
    """

    def __init__(self, player_name, npc_name, gold_reward):
        """
        Khởi tạo đối tượng BattleNPC.

        :param player_name: Tên của người chơi.
        :param npc_name: Tên của NPC.
        :param gold_reward: Số vàng thưởng khi thắng trận.
        """
        self.verify_player_and_npc_name_valid(player_name, npc_name)
        self.player = get_player_obj_by_name(player_name)
        self.npc = get_npc_obj_by_name(npc_name)
        self.gold_reward = int(gold_reward)
        self.start_time_utc = None

    def prompt_user_action(self):
        """
        Nhắc nhở người chơi chọn hành động trong trận chiến.

        :return: Hành động của người chơi ('a', 'i' hoặc 'q').
        """
        acceptable_actions = ["a", "q"] if not self.player.items else ["a", "i", "q"]
        prompt = "Do you want to Attack (a), Use an Item (i), or Quit (q): "
        user_action = input(prompt)

        while user_action not in acceptable_actions:
            print("Invalid option, acceptable options are:", acceptable_actions)
            user_action = input(prompt)

        return user_action

    def prompt_player_item(self, player_obj):
        """
        Hiển thị danh sách vật phẩm của người chơi và yêu cầu chọn vật phẩm để sử dụng.

        :param player_obj: Đối tượng người chơi.
        :return: Tên vật phẩm được chọn.
        """
        for index, item in enumerate(player_obj.items):
            print(f"[{index}] - {item}")

        item_index = int(input("Which item do you want to use?"))
        while item_index not in range(len(player_obj.items)):
            print("Invalid item index.")
            item_index = int(input("Which item do you want to use?"))

        return player_obj.items[item_index].name

    def execute_player_action(self, user_action):
        """
        Xử lý hành động của người chơi.

        :param user_action: Hành động của người chơi ('a' hoặc 'i').
        """
        if user_action == "a":
            self.player.attack(self.npc)
        elif user_action == "i":
            item_name = self.prompt_player_item(self.player)
            self.player.use_item(item_name)

    def npc_action(self):
        """
        Xử lý hành động của NPC.
        """
        npc_action = "a" if not self.npc.items else np.random.choice(["a", "i"], p=[0.7, 0.3])

        if npc_action == "i":
            item_index = np.random.randint(0, len(self.npc.items))
            self.npc.use_item(self.npc.items[item_index].name)
        else:
            self.npc.attack(self.player)

    def generate_battle_log_dict(self, result):
        """
        Tạo log trận đấu dưới dạng dictionary.

        :param result: Kết quả của trận đấu.
        :return: Dictionary chứa thông tin trận đấu.
        """
        return {
            "start_time_utc": self.start_time_utc,
            "player_name": self.player.name,
            "npc_name": self.npc.name,
            "result": result
        }

    def verify_player_and_npc_name_valid(self, player_name, npc_name):
        """
        Kiểm tra xem tên người chơi và NPC có hợp lệ không.

        :param player_name: Tên người chơi.
        :param npc_name: Tên NPC.
        """
        assert find_player_by_name(player_name) is not None, f"Player '{player_name}' không tồn tại."
        assert find_npc_by_name(npc_name) is not None, f"NPC '{npc_name}' không tồn tại."

    def start(self):
        """
        Bắt đầu trận đấu giữa người chơi và NPC.
        """
        self.start_time_utc = datetime.utcnow()
        round_num = 0

        while self.player.health > 0 and self.npc.health > 0:
            print(f"\n-- Round {round_num}. Player(health={self.player.health}, energy={self.player.energy}), "
                  f"NPC(health={self.npc.health}, energy={self.npc.energy})")

            user_action = self.prompt_user_action()
            if user_action == "q":
                print("Quitting battle...")
                battle_result = f"Player {self.player.name} quit against {self.npc.name}"
                insert_battle_log_by_dict(self.generate_battle_log_dict(battle_result))
                return None

            self.execute_player_action(user_action)
            if self.npc.health > 0:
                self.npc_action()

            update_player_by_obj(self.player)
            round_num += 1

        if self.player.health > 0 and self.npc.health <= 0:
            self.player.gold += self.gold_reward
            battle_result = f"Player {self.player.name} won the battle against {self.npc.name}"
        else:
            battle_result = f"Player {self.player.name} was beaten by {self.npc.name}"

        print(f"\n-- {battle_result} --")
        self.player.rest()
        update_player_by_obj(self.player)
        insert_battle_log_by_dict(self.generate_battle_log_dict(battle_result))


#### Use the functions and classes above to create a Player and an NPC, and start a battle between them

In [52]:
# Tạo NPC và người chơi với vật phẩm
npc_items = [Item("Bình máu", 2, [{"heal": 10}])]
npc = NPC("Goblin", 50, 25, npc_items)

player_items = [Item("Bình máu lớn", 2, [{"heal": 25}]),
                Item("Bình năng lượng lớn", 2, [{"restore_energy": 25}])]
player = Player("Seraphine", 60, 35, player_items)

# Hiển thị thông tin nhân vật
player.stats()

{'name': 'Seraphine',
 'health': 60,
 'max_health': 60,
 'energy': 35,
 'max_energy': 35,
 'items': [Item(name=Bình máu lớn, quantity=2, effects=[{'heal': 25}]),
  Item(name=Bình năng lượng lớn, quantity=2, effects=[{'restore_energy': 25}])],
 'gold': 0}

In [53]:
# 13) Insert Player into MongoDB
insert_player_by_obj(player)

ObjectId('67c41c1f897f4c58d6be937c')

In [55]:
# 14) Insert NPC into MongoDB
insert_npc_by_obj(npc)

InvalidDocument: Invalid document {'name': 'Goblin', 'health': 50, 'max_health': 50, 'energy': 25, 'max_energy': 25, 'items': [Item(name=Bình máu, quantity=2, effects=[{'heal': 10}])], 'gold': 0, '_id': ObjectId('67c41c97897f4c58d6be937e')} | cannot encode object: Item(name=Bình máu, quantity=2, effects=[{'heal': 10}]), of type: <class '__main__.Item'>

In [56]:
# 15) Start a battle between the Player and NPC you created using the BattleNPC class
#     Hint: After initalizing BattleNPC, you will need to call its start() method
battle = BattleNPC("Seraphine", "Goblin", 500)
battle.start()

AssertionError: NPC 'Goblin' không tồn tại.